# This script is used to compute the BIC values of each tree inferred in each model.

In [11]:
# This script is used to compute the BIC values of each tree inferred in each model.

import os
from os.path import join
import math
import csv
import pandas as pd

DIR_WORKING = '/Users/u7875558/Documents/RNAPhylo/allModels_SEED'
DIR_OUTPUTS = join(DIR_WORKING, 'outputs')
DIR_INPUTS = join(DIR_WORKING, 'inputs')
DIR_FASTA = join(DIR_INPUTS, 'fasta_files')

MODELS= [m for m in os.listdir(DIR_OUTPUTS) if m.startswith('S')]

# Number of free parameters:
K_MAP = {
    "DNAtrees": 5 + 3 +1,
    # 6 statespace
    "S6A": 14 + 5 + 1,
    "S6B":  2 + 5 + 1,
    "S6C":  2 + 2 + 1,
    "S6D":  1 + 2 + 1,
    "S6E":  1 + 5 + 1,
    # 7 statespace
    "S7A": 20 + 6 + 1,
    "S7B": 20 + 3 + 1,
    "S7C":  9 + 6 + 1,
    "S7D":  3 + 6 + 1,
    "S7E":  1 + 6 + 1,
    "S7F":  3 + 3 + 1,
    # 16 state space
    "S16":119 +15 + 1,
    "S16A": 4 +15 + 1,
    "S16B": 0 +15 + 1
}
SIG_RNAS = ['RF00740', 'RF00872', 'RF01038', 'RF02613', 'RF04290']

# regex to pull out
# - final log-likelihood 
# LH_RE = re.compile(r"Final.*Score.*:\s*(-?[0-9]+\.[0-9]+)")

# ────────────────────────────────────────────────────────────────────
def parse_fasta(fi_fasta):
    """
    Return (taxa_count, site_count) for a FASTA alignment
    """
    taxa = 0
    seq_line = ""
    first = False

    with open(fi_fasta) as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith('>'):
                taxa += 1
                if not first:
                    first = True
                    seq_line = ""
                elif first:
                    # we hit header for 2nd recrod -- done collecting the first one
                    continue
            else:
                if first:
                    seq_line = line.strip('\n')
    
    # 'seq_line' now hold the entire first sequence
    return taxa, len(seq_line)

rows = []
for model in K_MAP.keys():
    if model.startswith("DNA"):
        model_dir = join(DIR_OUTPUTS, model)
    else:
        model_dir = os.path.join(DIR_OUTPUTS, model, 'raxmlP_iPseu')
    
    if not os.path.isdir(model_dir):
        continue
    
    for rna in SIG_RNAS:
        rna_dir = join(model_dir, rna)
        if not os.path.isdir(rna_dir):
            continue

        # get taxa and sites from FASTA
        fasta_file = join(DIR_FASTA, f"{rna}.nodup.fa")
        if not os.path.isfile(fasta_file):
            print(f"Missing fasta file for {rna}")
            continue
        n_seq, n_sites = parse_fasta(fasta_file)

        # parse each replicate
        for fn in os.listdir(rna_dir):
            if not fn.startswith("RAxML_info."):
                continue
            seed = fn.split(".")[-1]
            logL = None

            with open(join(rna_dir, fn)) as inf:
                for line in inf.readlines():
                    if line.startswith("Final GAMMA"):
                        logL = float(line.split(" ")[-1])
                    if logL:
                        break

            #if logL is None:
            #    print(f"No log-likelihood is found in {fn}, {rna_dir}")
            #continue

            k_mod = K_MAP.get(model)
            k_br = 2 * n_seq - 3
            if model.startswith('DNA'):
                k_tot = k_mod + k_br  
            else:
                k_tot = k_mod + k_br + 9 # 9 is the number of free parameters of DNA.
            bic = -2 * logL + k_tot * math.log(n_sites)

            rows.append({
                "RNA":      rna,
                "model":    model,
                "seed":      seed,
                "logL":     logL,
                "n_seq":   n_seq,
                "n_sites":  n_sites,
                "k_mod":    k_mod,
                "k_br":     k_br,
                "k_tot":    k_tot,
                "BIC":      round(bic, 5)
            })

# build data frame
df = pd.DataFrame(rows)

df

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC
0,RF00740,DNAtrees,05,-161.203611,9,80,9,15,24,427.57586
1,RF00740,DNAtrees,02,-161.203618,9,80,9,15,24,427.57588
2,RF00740,DNAtrees,03,-161.203619,9,80,9,15,24,427.57588
3,RF00740,DNAtrees,04,-161.203533,9,80,9,15,24,427.57571
4,RF00740,DNAtrees,10,-161.203508,9,80,9,15,24,427.57566
...,...,...,...,...,...,...,...,...,...,...
745,RF04290,S16B,08,-124.680970,6,84,16,9,34,400.00971
746,RF04290,S16B,01,-124.680970,6,84,16,9,34,400.00971
747,RF04290,S16B,06,-124.680970,6,84,16,9,34,400.00971
748,RF04290,S16B,07,-124.680970,6,84,16,9,34,400.00971


# RF00740

In [13]:
df_00740 = df[df['RNA'] == 'RF00740']
df_00740

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC
0,RF00740,DNAtrees,05,-161.203611,9,80,9,15,24,427.57586
1,RF00740,DNAtrees,02,-161.203618,9,80,9,15,24,427.57588
2,RF00740,DNAtrees,03,-161.203619,9,80,9,15,24,427.57588
3,RF00740,DNAtrees,04,-161.203533,9,80,9,15,24,427.57571
4,RF00740,DNAtrees,10,-161.203508,9,80,9,15,24,427.57566
...,...,...,...,...,...,...,...,...,...,...
705,RF00740,S16B,01,-132.997793,9,80,16,15,40,441.27665
706,RF00740,S16B,06,-132.997793,9,80,16,15,40,441.27665
707,RF00740,S16B,08,-132.997793,9,80,16,15,40,441.27665
708,RF00740,S16B,09,-132.997793,9,80,16,15,40,441.27665


In [ ]:
# within each model, pick the replicate with highest logL
idx_ll = df_00740.groupby('model')['logL'].idxmax()
best_by_ll = df_00740.loc[idx_ll].reset_index(drop=True)

# tag each model's family: S6*, S7*, S16*
best_by_ll['group'] = best_by_ll['model'].str.extract(r'^(S6|S7|S16|DNA)')

best_by_ll

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF00740,DNAtrees,10,-161.203508,9,80,9,15,24,427.57566,DNA
1,RF00740,S16,03,-123.240636,9,80,135,15,159,943.22351,S16
2,RF00740,S16A,03,-126.046352,9,80,20,15,44,444.90188,S16
3,RF00740,S16B,05,-132.997793,9,80,16,15,40,441.27665,S16
4,RF00740,S6A,05,-123.103248,9,80,20,15,44,439.01567,S6
5,RF00740,S6B,05,-125.972227,9,80,8,15,32,392.16931,S6
6,RF00740,S6C,05,-128.973923,9,80,5,15,29,385.02662,S6
7,RF00740,S6D,05,-128.973449,9,80,4,15,28,380.64364,S6
8,RF00740,S6E,05,-125.971841,9,80,7,15,31,387.78651,S6
9,RF00740,S7A,05,-123.097809,9,80,27,15,51,469.67898,S7


In [16]:
# within each family with the same statespace, pick the one with lowest BIC 
idx_bic = best_by_ll.groupby('group')['BIC'].idxmin()
best_overall = best_by_ll.loc[idx_bic].reset_index(drop=True)

best_overall

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF00740,DNAtrees,10,-161.203508,9,80,9,15,24,427.57566,DNA
1,RF00740,S16B,05,-132.997793,9,80,16,15,40,441.27665,S16
2,RF00740,S6D,05,-128.973449,9,80,4,15,28,380.64364,S6
3,RF00740,S7E,04,-125.926667,9,80,8,15,32,392.07819,S7


# RF00872

In [17]:
df_00872 = df[df['RNA'] =='RF00872' ]

# within each model, pick the replicate with highest logL
idx_ll = df_00872.groupby('model')['logL'].idxmax()
best_by_ll = df_00872.loc[idx_ll].reset_index(drop=True)

# tag each model's family: S6*, S7*, S16*
best_by_ll['group'] = best_by_ll['model'].str.extract(r'^(S6|S7|S16|DNA)')

best_by_ll

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF00872,DNAtrees,07,-162.188207,8,82,9,13,22,421.32424,DNA
1,RF00872,S16,05,-113.856150,8,82,135,13,157,919.56722,S16
2,RF00872,S16A,09,-119.101858,8,82,20,13,42,423.28592,S16
3,RF00872,S16B,05,-132.825101,8,82,16,13,38,433.10553,S16
4,RF00872,S6A,05,-102.733906,8,82,20,13,42,390.55002,S6
5,RF00872,S6B,05,-104.089756,8,82,8,13,30,340.38109,S6
6,RF00872,S6C,05,-104.137905,8,82,5,13,27,327.25723,S6
7,RF00872,S6D,05,-104.136231,8,82,4,13,26,322.84716,S6
8,RF00872,S6E,05,-104.088118,8,82,7,13,29,335.97109,S6
9,RF00872,S7A,03,-114.377125,8,82,27,13,49,444.68349,S7


In [18]:
# within each family with the same statespace, pick the one with lowest BIC 
idx_bic = best_by_ll.groupby('group')['BIC'].idxmin()
best_overall = best_by_ll.loc[idx_bic].reset_index(drop=True)

best_overall

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF00872,DNAtrees,07,-162.188207,8,82,9,13,22,421.32424,DNA
1,RF00872,S16A,09,-119.101858,8,82,20,13,42,423.28592,S16
2,RF00872,S6D,05,-104.136231,8,82,4,13,26,322.84716,S6
3,RF00872,S7F,05,-117.885590,8,82,7,13,29,363.56604,S7


# RF01038

In [19]:
df_01038 = df[df['RNA'] =='RF01038' ]

# within each model, pick the replicate with highest logL
idx_ll = df_01038.groupby('model')['logL'].idxmax()
best_by_ll = df_01038.loc[idx_ll].reset_index(drop=True)

# tag each model's family: S6*, S7*, S16*
best_by_ll['group'] = best_by_ll['model'].str.extract(r'^(S6|S7|S16|DNA)')

best_by_ll

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF01038,DNAtrees,05,-137.765266,7,90,9,11,20,365.52673,DNA
1,RF01038,S16,02,-96.192598,7,90,135,11,155,889.85569,S16
2,RF01038,S16A,01,-97.508178,7,90,20,11,40,375.00874,S16
3,RF01038,S16B,06,-99.603832,7,90,16,11,36,361.20081,S16
4,RF01038,S6A,06,-95.947570,7,90,20,11,40,371.88753,S6
5,RF01038,S6B,01,-97.310536,7,90,8,11,28,320.61574,S6
6,RF01038,S6C,07,-99.439915,7,90,5,11,25,311.37507,S6
7,RF01038,S6D,07,-99.439749,7,90,4,11,24,306.87493,S6
8,RF01038,S6E,01,-97.310375,7,90,7,11,27,316.11561,S6
9,RF01038,S7A,07,-95.965590,7,90,27,11,47,403.42223,S7


In [20]:
# within each family with the same statespace, pick the one with lowest BIC 
idx_bic = best_by_ll.groupby('group')['BIC'].idxmin()
best_overall = best_by_ll.loc[idx_bic].reset_index(drop=True)

best_overall

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF01038,DNAtrees,05,-137.765266,7,90,9,11,20,365.52673,DNA
1,RF01038,S16B,06,-99.603832,7,90,16,11,36,361.20081,S16
2,RF01038,S6D,07,-99.439749,7,90,4,11,24,306.87493,S6
3,RF01038,S7F,10,-99.472269,7,90,7,11,27,320.43940,S7


# RF02613

In [21]:
df_02613 = df[df['RNA'] =='RF02613' ]

# within each model, pick the replicate with highest logL
idx_ll = df_02613.groupby('model')['logL'].idxmax()
best_by_ll = df_02613.loc[idx_ll].reset_index(drop=True)

# tag each model's family: S6*, S7*, S16*
best_by_ll['group'] = best_by_ll['model'].str.extract(r'^(S6|S7|S16|DNA)')

best_by_ll

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF02613,DNAtrees,07,-240.911263,4,160,9,5,14,552.87496,DNA
1,RF02613,S16,09,-167.054037,4,160,135,5,149,1090.30897,S16
2,RF02613,S16A,09,-167.054573,4,160,20,5,34,506.66506,S16
3,RF02613,S16B,09,-169.317464,4,160,16,5,30,490.89014,S16
4,RF02613,S6A,09,-167.410095,4,160,20,5,34,507.37610,S6
5,RF02613,S6B,09,-168.627102,4,160,8,5,22,448.90803,S6
6,RF02613,S6C,09,-173.246072,4,160,5,5,19,442.92045,S6
7,RF02613,S6D,09,-173.246027,4,160,4,5,18,437.84518,S6
8,RF02613,S6E,09,-168.627059,4,160,7,5,21,443.83277,S6
9,RF02613,S7A,06,-166.589011,4,160,27,5,41,541.26015,S7


In [22]:
# within each family with the same statespace, pick the one with lowest BIC 
idx_bic = best_by_ll.groupby('group')['BIC'].idxmin()
best_overall = best_by_ll.loc[idx_bic].reset_index(drop=True)

best_overall

,RNA,model,seed,logL,n_seq,n_sites,k_mod,k_br,k_tot,BIC,group
0,RF02613,DNAtrees,07,-240.911263,4,160,9,5,14,552.87496,DNA
1,RF02613,S16B,09,-169.317464,4,160,16,5,30,490.89014,S16
2,RF02613,S6D,09,-173.246027,4,160,4,5,18,437.84518,S6
3,RF02613,S7E,09,-167.678444,4,160,8,5,22,447.01071,S7
